In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!pip install transformers datasets peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 27.4 MB/s eta 0:00:00


In [14]:
from datasets import load_dataset

In [15]:
data = load_dataset("Fredithefish/Instruction-Tuning-with-GPT-4-RedPajama-Chat")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [16]:
df = data["train"].to_pandas()
df

,text
0,<human>: Give three tips for staying healthy.\...
1,<human>: What are the three primary colors?\n<...
2,<human>: Describe the structure of an atom.\n<...
3,<human>: How can we reduce air pollution?\n<bo...
4,<human>: Describe a time when you had to make ...
...,...
51997,<human>: Generate an example of what a resume ...
51998,<human>: Arrange the items given below in the ...
51999,<human>: Write an introductory paragraph about...
52000,<human>: Generate a list of five things one sh...


In [17]:
df["text"][0]

'<human>: Give three tips for staying healthy.\n<bot>: 1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.'

In [18]:
df["text"][0].split("<bot>:")[0]

'<human>: Give three tips for staying healthy.\n'

In [19]:
df["Assistant"] = df["text"].apply(lambda x: x.split("<bot>:")[-1])
df["Human"] = df["text"].apply(lambda x: "Human:" + x.split("<bot>:")[0].replace("<human>:", "").replace("\n", "") )
df

,text,Assistant,Human
0,<human>: Give three tips for staying healthy.\...,1. Eat a balanced and nutritious diet: Make s...,Human: Give three tips for staying healthy.
1,<human>: What are the three primary colors?\n<...,"The three primary colors are red, blue, and y...",Human: What are the three primary colors?
2,<human>: Describe the structure of an atom.\n<...,An atom is the basic building block of all ma...,Human: Describe the structure of an atom.
3,<human>: How can we reduce air pollution?\n<bo...,There are several ways to reduce air pollutio...,Human: How can we reduce air pollution?
4,<human>: Describe a time when you had to make ...,"As an AI assistant, I do not have my own pers...",Human: Describe a time when you had to make a ...
...,...,...,...
51997,<human>: Generate an example of what a resume ...,**John Doe**\n\n**Contact Information:**\n\n1...,Human: Generate an example of what a resume sh...
51998,<human>: Arrange the items given below in the ...,Me eating cake.,Human: Arrange the items given below in the or...
51999,<human>: Write an introductory paragraph about...,"Michelle Obama, born January 17, 1964, in Chi...",Human: Write an introductory paragraph about a...
52000,<human>: Generate a list of five things one sh...,1. Evaluate your reasons: It's important to t...,Human: Generate a list of five things one shou...


In [20]:
df = df[["Human", "Assistant"]]
df

,Human,Assistant
0,Human: Give three tips for staying healthy.,1. Eat a balanced and nutritious diet: Make s...
1,Human: What are the three primary colors?,"The three primary colors are red, blue, and y..."
2,Human: Describe the structure of an atom.,An atom is the basic building block of all ma...
3,Human: How can we reduce air pollution?,There are several ways to reduce air pollutio...
4,Human: Describe a time when you had to make a ...,"As an AI assistant, I do not have my own pers..."
...,...,...
51997,Human: Generate an example of what a resume sh...,**John Doe**\n\n**Contact Information:**\n\n1...
51998,Human: Arrange the items given below in the or...,Me eating cake.
51999,Human: Write an introductory paragraph about a...,"Michelle Obama, born January 17, 1964, in Chi..."
52000,Human: Generate a list of five things one shou...,1. Evaluate your reasons: It's important to t...


In [21]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.2, shuffle=True, random_state=0)
train_data

,Human,Assistant
34452,Human: Give three reasons why long-term planni...,1) Anticipating and Preempting Future Challen...
11969,Human: Sort the following terms based on their...,"Helium, Lithium, Uranium."
26533,Human: Identify the appropriate example of a v...,"""He eats breakfast every morning."" In this se..."
41946,Human: Copy the following text while making ne...,Jane is a driven and resolute student who con...
44683,Human: Compute the following: 2+3,The result of the computation is 5.
...,...,...
21243,Human: Perform a Google search and find a webs...,"After conducting a Google search, I found a w..."
45891,"Human: Compose five sentences using the verb ""...",1. John sat quietly in the garden as he conte...
42613,Human: Emphasize one word in this sentenceHe j...,He *just* wanted to be heard.
43567,Human: What year did the Great Recession start...,The Great Recession started in the United Sta...


In [22]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

In [23]:
from peft import get_peft_model, LoraConfig, TaskType
import pandas as pd
from datasets import Dataset

In [24]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41601 entries, 34452 to 2732
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Human      41601 non-null  object
 1   Assistant  41601 non-null  object
dtypes: object(2)
memory usage: 975.0+ KB


In [25]:
train_data = Dataset.from_pandas(train_data)
test_data = Dataset.from_pandas(test_data)

In [26]:
train_data

Dataset({
    features: ['Human', 'Assistant', '__index_level_0__'],
    num_rows: 41601
})

In [27]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

In [31]:
def preprocess_function(sample,padding=True):
    model_inputs = tokenizer(sample["Human"], max_length=256, padding=padding, truncation=True)
    labels = tokenizer(sample["Assistant"], max_length=256, padding=padding, truncation=True)
    if padding == True:
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [29]:
train_tokenized_dataset = train_data.map(preprocess_function, batched=True, remove_columns=train_data.column_names)
test_tokenized_dataset = test_data.map(preprocess_function, batched=True, remove_columns=test_data.column_names)
print(f"Keys of tokenized dataset: {list(train_tokenized_dataset.features)}")

Map:   0%|          | 0/41601 [00:00<?, ? examples/s]

Map:   0%|          | 0/10401 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [32]:
lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.1,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)

In [33]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 4,718,592 || all params: 787,868,672 || trainable%: 0.5989059049678777


In [34]:
label_pad_token_id = -100
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [35]:
from huggingface_hub import notebook_login
notebook_login()

In [36]:
output_dir="lora-flan-t5-large-chat"
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    learning_rate=1e-3,
    num_train_epochs=1,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="epoch",
    save_strategy="epoch",
    report_to="tensorboard",
    push_to_hub = True
)

In [37]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_tokenized_dataset,
)
model.config.use_cache = False

trainer.train()
peft_save_model_id="lora-flan-t5-large-chat"
trainer.model.save_pretrained(peft_save_model_id, push_to_hub=True)
tokenizer.save_pretrained(peft_save_model_id, push_to_hub=True)
trainer.model.base_model.save_pretrained(peft_save_model_id, push_to_hub=True)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10401,2.006100


pytorch_model.bin:   0%|          | 0.00/3.15G [00:00<?, ?B/s]

In [38]:
! cp -r /content/lora-flan-t5-large-chat/ /content/drive/MyDrive/Chatbot/

In [39]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
peft_model_id = "lora-flan-t5-large-chat"
config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0}).cuda()
model.eval()

sample = "Human: \nExplain me about the working of Artificial Intelligence. \nAssistant: "
input_ids = tokenizer(sample, return_tensors="pt", truncation=True, max_length=256).input_ids.cuda()
outputs = model.generate(input_ids=input_ids, do_sample=True, top_p=0.9, max_length=256)
print(f"{sample}")

print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

Human: 
Explain me about the working of Artificial Intelligence. 
Assistant: 
Artificial intelligence (AI) is a field that focuses on developing computer systems that can act like humans. It involves modifying or enhancing existing human abilities, such as problem-solving, decision-making, or language-learning. Some of the core activities of AI include: 1. Data collection: AI is used to analyze and analyze data for patterns, trends, and information. This includes using machine learning algorithms to process and analyze large amounts of data and to make decisions about the future. 2. Model-building: AI helps computer systems understand human relationships, behaviors, and motivations. This can include developing artificial intelligence models based on human data, such as images or voice recognition. 3. Training and learning: AI allows computer systems to learn from experience and make recommendations for further learning. This can include using a network of machine learning algorithms to